# Parte 3 -  Criação de Features

## Objetivos:

Na terceira etapa do desafio, o que será explorado será exatamente a combinação de features e o uso de informações externas.

No final dessa etapa, será treinado um modelo agregando todo o pipeline desenvolvido até o momento.

# Setup do Ambiente

## Magic Functions do Jupyter

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

## Imports de Libs Externas (padrão)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import numpy as np
import os
import pandas as pd

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib

## Imports de Libs Locais

In [ ]:
from dataset import load_california_housing_prices
from pipeline import (
    ManuallyCraftedFeaturesTransform, PolynomialFeaturesTransform, 
    PointsOfInterestFeaturesTransform, FeaturesChoiceTransform
)

## Carregando o Dataset:

In [ ]:
dataset = load_california_housing_prices()
x_train = dataset["train"]["x"]
y_train = dataset["train"]["y"]
x_test = dataset["test"]["x"]
y_test = dataset["test"]["y"]

Como a Feature Engineering apresentada nessa etapa deve ser feita sobre as features já tratadas, as soluções numérica e categórica serão aplicadas aos dados originais. 

In [ ]:
prev_pipeline = Pipeline([
    ("numerical_feat_eng",   joblib.load(os.path.join("pipelines", "numerical_feat_eng.pkl"))),
    ("categorical_feat_eng", joblib.load(os.path.join("pipelines", "categorical_feat_eng.pkl")))
])

In [ ]:
x_train = prev_pipeline.transform(x_train)
x_test = prev_pipeline.transform(x_test)

# Features Engineering c/ Combinação de Features e Features Externas

Para enriquecer ainda mais os modelos, pode-se combinar as features existentes entre si e com features extraídas de fontes externas de conhecimento.

## Combinação Manual de Features

É uma prática comum agrupar manualmente features por assunto ou grupos de conhecimento, principalmente quando se tem um bom conhecimento do domínio de aplicação. 

Algumas features numéricas nesse dataset são bem relacionadas entre si, sendo imediato pensar em combiná-las. Para manter o padrão no pré-processamento, isso será feito em uma classe de Feature Transformer.

--------------
#### Tarefa (3.1) 

Completar a implementação do transformador de dados `ManuallyCraftedFeaturesTransform`. 

A classe está no arquivo `pipeline.py`.

---------------

Verificando as novas features criadas:

In [ ]:
ManuallyCraftedFeaturesTransform().transform(x_train).head(10)

## Combinação Polinomial de Features

Uma outra técnica muito utilizada em feature engineering é criar novas features a partir da combinação polinomial de features antigas. Dessa forma, uma solução linear pode ser extendida para uma solução não linear.

--------------
#### Tarefa (3.2) 

Completar a implementação do transformador de dados `PolynomialFeaturesTransform`. 

A classe está no arquivo `pipeline.py`.

---------------

A seguir pode-se observar um teste com uma amostra das features que compõem o dataset:

In [ ]:
features = [
    "longitude", "latitude", 
    "households", "total_bedrooms", "total_rooms"    
]

PolynomialFeaturesTransform(features, 3).fit_transform(x_train).head(10)

## Dados Externos: Pontos de Interesse

Uma das técnicas que mais trazem informação para o modelo é a inclusão de dados externos. 

Para essa seção, deve-se escolher alguns pontos de interesse da Califórina e arredores para calcular a distância euclidiana da Latitude/Longitude do data point a esses pontos de interesse.

--------------
#### Tarefa (3.3) 

Completar a implementação do transformador de dados `PointsOfInterestFeaturesTransform`. 

A classe está no arquivo `pipeline.py`.

---------------

A seguir o novo transformador pode ser visto em ação:

In [ ]:
PointsOfInterestFeaturesTransform().fit_transform(x_test.head())

## Treinamento e Avaliação de um Modelo Linear

Serão utilizadas todas as soluções construídas de Feature Engineering para extrair ao máximo o potencial do modelo linear.

###  Reload das massas de Treino e de Teste

As massas de dados de Treino e de Teste serão carregadas novamente para que seja aplicado o pipeline de pré-processamento em ambos desde o princípio. 

In [ ]:
dataset = load_california_housing_prices()
x_train = dataset["train"]["x"]
y_train = dataset["train"]["y"]
x_test = dataset["test"]["x"]
y_test = dataset["test"]["y"]

Deve-se remover os outliers da massa de treino usando a função construída para isso.

In [ ]:
keep_index = joblib.load(os.path.join("pipelines", "keep_index.pkl"))
x_train = x_train[keep_index]
y_train = y_train[keep_index]

###  Pipeline contendo todas as Feature Engineerings construídas até o momento


####  Pipeline de Pré-Processamento

Todas as etapas de pré-processamento devem estar incluídas nesse pipeline.

In [ ]:
numerical_features = [
    "longitude", "latitude", 
    "housing_median_age", 
    "total_rooms", "total_bedrooms",  
    "population", "households", "median_income"
]

log_transform_features = [
    "total_rooms", "total_bedrooms", 
    "population", "households", 
    "median_income"
]

categories = ["<1H OCEAN", "INLAND", "NEAR OCEAN", "NEAR BAY"]

poly_features = [
    "longitude", "latitude", 
     "housing_median_age", "total_rooms", "total_bedrooms",
     "population", "households", "median_income",
     "log_of_total_rooms", "log_of_total_bedrooms", "log_of_population", 
     "log_of_households", "log_of_median_income"
]
poly_degree = 3

In [ ]:
chosen_features = (numerical_features +
                   [f"log_of_{c}" for c in log_transform_features] + 
                   [f"ocean_proximity: {c}" for c in categories])

In [ ]:
pipeline = Pipeline([
    ("previous_pipeline",          prev_pipeline),
    ("features_choice",            FeaturesChoiceTransform(chosen_features)),
    ("manually_crafted_transform", ManuallyCraftedFeaturesTransform()),
    ("polynomial_feats_transform", PolynomialFeaturesTransform(poly_features, poly_degree)),
    ("ext_poi_feats_transform",    PointsOfInterestFeaturesTransform()),
    ("zscore",                     StandardScaler()),
    ("predictor",                  ElasticNet()),
])

####  Treinar e avaliar o modelo

In [ ]:
pipeline.fit(x_train, y_train)

Avaliação do modelo nas massas de **treino** e de **teste**.

In [ ]:
y_true = y_train
y_pred = pipeline.predict(x_train)
mse_tr = mean_squared_error(y_true=y_true, y_pred=y_pred)
r2_tr = r2_score(y_true=y_true, y_pred=y_pred)

In [ ]:
y_true = y_test
y_pred = pipeline.predict(x_test)
mse_te = mean_squared_error(y_true=y_true, y_pred=y_pred)
r2_te = r2_score(y_true=y_true, y_pred=y_pred)

In [ ]:
pd.DataFrame(
    index=["train", "test"],
    columns=["MSE", "R^2"],
    data=[
        [mse_tr, r2_tr],
        [mse_te, r2_te]
    ]
)